<a href="https://colab.research.google.com/github/YellaCRE/FIFA_RecommandSystem/blob/main/FIFA_modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#피파온라인4 선수추천알고리즘_추천 모델

#데이터 불러오기

In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity
from pprint import pprint
from numpy import empty
import numpy as np
from numpy.ma.core import mean
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import minmax_scale
import sys
pd.set_option('mode.chained_assignment',  None)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

data = pd.read_csv("/content/drive/MyDrive/2022.2 피파선수 추천 프로젝트/player_num_total.csv")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
data[['개인기','right','left']] = data[['개인기','right','left']]/20

#선수 입력

In [ ]:
#허용 데이터 범위
ran = 2 #숫자가 클수록 낮은 스탯의 선수도 허용함

In [ ]:
using_df_list = pd.DataFrame()
for i in ['첫번째','두번째','세번째','네번째']:
  #사용하는 선수 이름 검색
  print("사용하는 "+str(i)+" 선수 이름을 입력하세요(ex.호날두, 메시) : ")
  while True:
    using_name = input()
    if (data['name'].str.contains(using_name)==True).any() :
      using_df = data[data['name'].str.contains(using_name)].sort_values(by = '스탯',ascending=False)
      if using_df.empty:
        print("해당 선수가 존재하지 않습니다")
        sys.exit()
      break
      #print(using_df)
    else:
      print("이름을 다시 확인해주세요")

  if using_name == '':
    print("입력을 종료합니다")
    break
    
  #사용하는 선수 시즌
  print("시즌을 선택하세요(ex.ICON, BTB, TT) : ")
  while True:
    season = input().upper()
    if str(season) in data['시즌'].unique() :
      using_df = using_df[using_df['시즌'].str.contains(season, na=False)].sort_values(by = '스탯',ascending=False)
      if using_df.empty:
        print("해당 선수가 존재하지 않습니다")
        sys.exit()
      #print(using_df)
      break
    else:
      print("시즌을 다시 확인해주세요")


  #강화 수치
  print("강화 수치를 선택하세요(ex.1,5) : ")
  while True:
    plus = int(input())
    if plus in using_df['강화'].values.tolist() :
      using_df = using_df[using_df['강화'] == plus]
      if using_df.empty:
        print("해당 선수가 존재하지 않습니다")
        sys.exit()
      #print(using_df)
      break
    else:
      print("강화수치를 다시 확인해주세요")
  
  print("\n")
  using_df_list = pd.concat([using_df_list,using_df])

print(using_df_list[['id','name','시즌','포지션','스탯']])

사용하는 첫번째 선수 이름을 입력하세요(ex.호날두, 메시) : 
유상철
시즌을 선택하세요(ex.ICON, BTB, TT) : 
TKL
강화 수치를 선택하세요(ex.1,5) : 
7


사용하는 두번째 선수 이름을 입력하세요(ex.호날두, 메시) : 
굴리트
시즌을 선택하세요(ex.ICON, BTB, TT) : 
CFA
강화 수치를 선택하세요(ex.1,5) : 
5


사용하는 세번째 선수 이름을 입력하세요(ex.호날두, 메시) : 
고레츠카
시즌을 선택하세요(ex.ICON, BTB, TT) : 
22TOTN
강화 수치를 선택하세요(ex.1,5) : 
5


사용하는 네번째 선수 이름을 입력하세요(ex.호날두, 메시) : 
포그바
시즌을 선택하세요(ex.ICON, BTB, TT) : 
BTB
강화 수치를 선택하세요(ex.1,5) : 
5


               id     name      시즌 포지션   스탯
313827  225140053      유상철     TKL  CM  111
213414  254214100   루드 굴리트     CFA  CM  111
213758  259209658  레온 고레츠카  22TOTN  CM  111
213572  256195864    폴 포그바     BTB  CM  110


유상철, TKL, 7강

루드 굴리트, CFA, 5강

레온 고레츠카, 22TOTN, 5강

폴 포그바, BTB, 5강

개인기, 특성, 주발 설정

In [ ]:
#개인기 설정
print("최소 개인기요구치를 입력해주세요(필수) ex) 4 = 4성 이상")
while True :
    least_star = input()
    try :
        least_star = int(least_star)
        break
    except:
        print("다시 입력해주세요")

#주발 설정
print("주발을 선택해주세요 상관없으면 엔터 ex) 왼, 오")
while True :
    main_foot = str(input())
    if main_foot in ['왼','오',''] : break
    else:
      print("다시 입력해주세요")

#약발 설정  
print("최소 약발요구치를 입력해주세요(필수) ex) 4 = 약발 4 이상")
while True :
    least_weak = input()
    try :
        least_weak = int(least_weak)
        break
    except:
        print("다시 입력해주세요")

#특성 설정
print("필수 특성이 있다면 입력해주세요 종료하려면 엔터\nex)'승부욕', '아웃사이드슈팅/크로스','예리한감아차기', '주발선호', '파워헤더', '패스마스터', '화려한개인기', '화려한걷어내기'")
hidden_list = []
while True :
    hidden = str(input())
    if hidden == '':
      break
    if hidden in ['승부욕', '아웃사이드슈팅/크로스','예리한감아차기', '주발선호', '파워헤더', '패스마스터', '화려한개인기', '화려한걷어내기'] : 
      hidden_list.append(hidden)
    else:
      print("다시 입력해주세요")

최소 개인기요구치를 입력해주세요(필수) ex) 4 = 4성 이상
3
주발을 선택해주세요 상관없으면 엔터 ex) 왼, 오

최소 약발요구치를 입력해주세요(필수) ex) 4 = 약발 4 이상
4
필수 특성이 있다면 입력해주세요 종료하려면 엔터
ex)'승부욕', '아웃사이드슈팅/크로스','예리한감아차기', '주발선호', '파워헤더', '패스마스터', '화려한개인기', '화려한걷어내기'



#데이터 전처리

가중치 설정

In [ ]:
list_allstats = ['속력', '가속력','골 결정력', '슛 파워', '중거리 슛', '위치 선정', '발리 슛', '페널티 킥', '짧은 패스', '시야','크로스', '긴 패스', '프리킥', '커브', '드리블', '볼 컨트롤', '민첩성', '밸런스', '반응 속도','대인 수비', '태클', '가로채기', '헤더', '슬라이딩 태클', '몸싸움', '스태미너', '적극성', '점프','침착성', 'GK 다이빙', 'GK 핸들링', 'GK 킥', 'GK 반응속도', 'GK 위치선정']
list_player = ['속력', '가속력','골 결정력', '슛 파워', '중거리 슛', '위치 선정', '발리 슛', '페널티 킥', '짧은 패스', '시야','크로스', '긴 패스', '프리킥', '커브', '드리블', '볼 컨트롤', '민첩성', '밸런스', '반응 속도','대인 수비', '태클', '가로채기', '헤더', '슬라이딩 태클', '몸싸움', '스태미너', '적극성', '점프','침착성']
list_v = ['속력', '가속력','골 결정력', '슛 파워', '중거리 슛', '짧은 패스', '시야','긴 패스', '드리블', '볼 컨트롤', '민첩성', '밸런스', '반응 속도','태클', '가로채기', '헤더', '몸싸움', '스태미너', '적극성', '점프', '침착성']

#입력치로 받게 만들기
player_p=using_df_list[list_player]
sum=player_p.sum(axis=1)

for i in [0,1,2,3]:
    player_p.iloc[i]=player_p.iloc[i]/sum.iloc[i]

player_p.loc['평균'] = np.zeros((1,len(player_p.columns)))[0]

for m in player_p.columns:
    player_p.at['평균',m]=player_p[m].sum()/4

avg_p=pd.DataFrame(player_p.loc['평균'])
avg_p.loc[:,'norm']=minmax_scale(avg_p, feature_range = (0.87,1))
avg_p = avg_p.transpose()
weight = avg_p.loc['norm']

data[list_player] = round(data[list_player]*weight)

데이터 분리

In [ ]:
data_GK = data[data.포지션 == 'GK']
isB = data['포지션'].str.contains("B") #수비수
isM = data['포지션'].str.contains("M") #미드필더
isF = data['포지션'].str.contains("F|ST|RW|LW") #공격수 #다음코드에는 LWB와 RWB도 포함되지만 윙백의 경우도 윙으로 사용이 가능하기 때문에 허용 

data_B = data[isB]
data_M = data[isM]
data_F = data[isF]

#추천 모델

In [ ]:
#이름,시즌,강화수치로 검색
def search_name(using_data):
  using_name = using_data['name']
  using_season = using_data['시즌']
  using_plus = using_data['강화']

  using_df = data[data['name'].str.contains(using_name)]
  using_df = using_df[using_df['시즌'].str.contains(using_season, na=False)]
  using_df = using_df[using_df['강화'] == using_plus]
  
  using_ID = using_df.loc[:,'id']
  using_pois = using_df.loc[:,'포지션']
  using_stats = using_df.loc[:,'스탯']

  #포지션별 데이터 접근
  if using_pois.str.contains("F|ST|RW|LW").iloc[0] == True : 
    temp_data = data_F
  if using_pois.str.contains("M").iloc[0] == True : 
    temp_data = data_M
  if using_pois.str.contains("D").iloc[0] == True : 
    temp_data = data_B

  #스탯에 따른 데이터 접근
  temp_data = temp_data[temp_data['스탯'] >= int(using_stats)-ran]
  temp_data = temp_data[temp_data['스탯'] <= int(using_stats)+ran]

  #인덱스 재구성
  del temp_data['index']
  temp_data.insert(0,'index',range(len(temp_data)))

  temp_num = temp_data[['키', '몸무게', '체형']]

  temp_cos = cosine_similarity(temp_num, temp_num)
  recommand_cos = temp_cos[temp_data[temp_data['id'] == using_ID.iloc[0]]['index'].iloc[0],]

  #코사인 유사도 표기
  temp_data['cos'] = recommand_cos

  #추천리스트의 길이를 설정가능
  i = 0
  print("조건에 맞는 "+str(len(temp_data))+'명의 선수를 검색합니다')

  for n in range(len(using_df_list)):
    i += len(temp_data[((temp_data['name']==using_df_list['name'].iloc[n]) & (temp_data['시즌']==using_df_list['시즌'].iloc[n]))])
    temp_data = temp_data[((temp_data['name'] != using_df_list['name'].iloc[n]) | (temp_data['시즌'] != using_df_list['시즌'].iloc[n]))]
  
  print('동일선수 '+str(i)+'명 제외')

  return temp_data #가장 첫번째 나오는 항목이 검색한 선수

In [ ]:
#선수 데이터베이스로 검색(1.끔찍한 혼종용 검색기)
def search_stat(using_df_list):
  temp1 = using_df_list.iloc[0][['index', 'id', 'name', '시즌', '포지션']]
  temp2 = np.round(using_df_list[['키', '몸무게', '스탯', '속력', '가속력',
       '골 결정력', '슛 파워', '중거리 슛', '발리 슛', '짧은 패스', '시야',
       '크로스', '긴 패스', '커브', '드리블', '볼 컨트롤', '민첩성', '밸런스', '반응 속도',
       '대인 수비', '태클', '가로채기', '헤더', '슬라이딩 태클', '몸싸움', '스태미너', '적극성', '점프',
       '침착성', '개인기', 'left', 'right', '체형', '승부욕', '아웃사이드슈팅/크로스',
       '예리한감아차기', '주발선호', '파워헤더', '패스마스터', '화려한개인기', '화려한걷어내기']].mean())
  temp3 =using_df_list.iloc[0][['강화']]

  using_df = pd.DataFrame()
  temp2 = temp2.append(temp3)
  temp1 = temp1.append(temp2)
  using_df = pd.DataFrame(temp1).transpose()

  using_pois = using_df.loc[:,'포지션']
  using_stats = using_df.loc[:,'스탯']
  name = using_df.loc[:,'name'][0]

  #포지션별 데이터 접근
  if using_pois.str.contains("F|ST|RW|LW").iloc[0] == True : 
    temp_data = data_F
  if using_pois.str.contains("M").iloc[0] == True : 
    temp_data = data_M
  if using_pois.str.contains("D").iloc[0] == True : 
    temp_data = data_B

  #스탯에 따른 데이터 접근
  temp_data = temp_data[temp_data['스탯'] >= int(using_stats)-ran]
  temp_data = temp_data[temp_data['스탯'] <= int(using_stats)+ran]

  #인덱스 재구성
  temp_data = pd.concat([using_df,temp_data])
  del temp_data['index']
  temp_data.insert(0,'index',range(len(temp_data)))

  temp_num = temp_data[['키', '몸무게', '체형']]

  temp_cos = cosine_similarity(temp_num, temp_num)
  recommand_cos = temp_cos[0,]

  #코사인 유사도 표기
  temp_data['cos'] = recommand_cos

  #추천리스트의 길이를 설정가능
  i = 0
  print("조건에 맞는 "+str(len(temp_data))+'명의 선수를 검색합니다')

  for n in range(len(using_df_list)):
    i += len(temp_data[((temp_data['name']==using_df_list['name'].iloc[n]) & (temp_data['시즌']==using_df_list['시즌'].iloc[n]))])
    temp_data = temp_data[((temp_data['name'] != using_df_list['name'].iloc[n]) | (temp_data['시즌'] != using_df_list['시즌'].iloc[n]))]
  
  print('동일선수 '+str(i)+'명 제외')

  return temp_data #가장 첫번째 나오는 항목이 검색한 선수

개인기, 주발약발, 특성, 중복제거 필터

In [ ]:
def filter(using_data):
  #개인기
  using_data = using_data[using_data['개인기'] >= least_star]

  #주발
  if main_foot == '오':
    using_data = using_data[using_data['right'] == 5]
    using_data = using_data[using_data['left'] >= least_weak]
  if main_foot == '왼':
    using_data = using_data[using_data['left'] == 5]
    using_data = using_data[using_data['right'] >= least_weak]
  if main_foot == '':
    using_data = using_data[using_data['left'] >= least_weak]
    using_data = using_data[using_data['right'] >= least_weak]
  
  #특성 ['승부욕', '아웃사이드슈팅/크로스','예리한감아차기', '주발선호', '파워헤더', '패스마스터', '화려한개인기', '화려한걷어내기']
  for hidden in hidden_list:
    if hidden in ['승부욕', '아웃사이드슈팅/크로스','예리한감아차기', '주발선호', '파워헤더', '패스마스터', '화려한개인기', '화려한걷어내기']:
      using_data = using_data[using_data[hidden] > 0]

  #비율로 스탯 변환
  num = using_data[list_player]
  sum = num.sum(axis=1)
  for i in range(len(num)):
    num.iloc[i]=num.iloc[i]/sum.iloc[i]
  using_data[list_player] = num

  #my_data 비율로 변환
  my_num = using_df_list[list_player]
  my_sum = my_num.sum(axis=1)
  for i in range(len(my_num)):
    my_num.iloc[i]=my_num.iloc[i]/my_sum.iloc[i]

  #하한 설정
  my_min = my_num.min(axis=0)
  my_min = my_min[list_v]

  #타겟 스탯으로 먼저 출력
  target_stat = round(using_df_list['스탯'].mean())
  using_data['target'] = ran - abs(target_stat - using_data['스탯']) #스탯 차이를 절댓값으로 바꾸고 스탯차이가 많이 날 수록 값이 낮다
  using_data = using_data.sort_values(['target','스탯','cos'],ascending=False) # 스탯차이, 스탯, 코사인유사도 순으로 소팅

  adj=0
  target = 1
  result_data = pd.DataFrame()
  while True:
    temp_data = using_data
    limit = my_min-adj

    for i in range(len(limit)):
      temp_data = temp_data[temp_data[limit.index[i]]>=limit[i]]
    
    #강화 수치만 다른 중복 제거
    temp_data = temp_data.drop_duplicates(['name','시즌'], keep = 'first')
    temp_data = temp_data.sort_values(['cos'],ascending=False)
    #1,2,3군 표시
    temp_data['우선순위'] = target

    #결과 데이터 저장
    result_data = pd.concat([result_data,temp_data])
    result_data = result_data.drop_duplicates(['name','시즌'], keep = 'first')

    #하한 낮추기
    adj+=0.0005 #하한 조정치
    target+=1
    if len(temp_data) >= 50:
      using_data = result_data
      break
    
  #코사인 유사도 필터
  using_data = using_data[using_data['cos']>=0.9997] #코사인 조정치
  using_data.reset_index(inplace=True)
  
  return using_data[['우선순위','name', '시즌', '포지션','키', '몸무게', '스탯','강화','cos']].head(10)

#추천리스트 출력

In [ ]:
for num in range(len(using_df_list)):
    print(using_df_list.iloc[num]['name']+" 선수에 대한 추천 리스트")
    print(filter(search_name(using_df_list.iloc[num])))
    print('\n')

유상철 선수에 대한 추천 리스트
조건에 맞는 2805명의 선수를 검색합니다
동일선수 9명 제외
   우선순위      name      시즌  포지션    키  몸무게   스탯  강화       cos
0     2        사울   19UCL   CM  184   77  110   9  0.999991
1     2   레온 고레츠카  19TOTS   CM  189   79  111   9  0.999947
2     3       유상철     CAP   CM  184   78  112   7  1.000000
3     3        사울  20TOTS   CM  184   76  111   7  0.999963
4     3        사울   20UCL   CM  184   75  112  10  0.999917
5     4  마리오 파샬리치   BOE21   CM  188   82  110  10  0.999909
6     4  마누엘 로카텔리  22TOTN  CDM  185   75  112   6  0.999896
7     4   레온 고레츠카   19PLS   CM  189   76  112  10  0.999830
8     4  E. 팔라시오스    20NG   CM  177   72  111   9  0.999767


루드 굴리트 선수에 대한 추천 리스트
조건에 맞는 2805명의 선수를 검색합니다
동일선수 9명 제외
   우선순위       name     시즌  포지션    키  몸무게   스탯  강화       cos
0     3     루드 굴리트    NHD   CM  191   88  110  10  1.000000
1     3     루드 굴리트     TT   CM  191   88  109   9  1.000000
2     4     루드 굴리트    HOT   CM  191   88  113   8  1.000000
3     4  슈테판 에펜베르크   ICON   CM  188   88  112   5

In [ ]:
#첫번째 선수 추천리스트
filter(search_name(using_df_list.iloc[0]))

조건에 맞는 2805명의 선수를 검색합니다
동일선수 9명 제외


,우선순위,name,시즌,포지션,키,몸무게,스탯,강화,cos
0,2,사울,19UCL,CM,184,77,110,9,0.999991
1,2,레온 고레츠카,19TOTS,CM,189,79,111,9,0.999947
2,3,유상철,CAP,CM,184,78,112,7,1.000000
3,3,사울,20TOTS,CM,184,76,111,7,0.999963
4,3,사울,20UCL,CM,184,75,112,10,0.999917
5,4,마리오 파샬리치,BOE21,CM,188,82,110,10,0.999909
6,4,마누엘 로카텔리,22TOTN,CDM,185,75,112,6,0.999896
7,4,레온 고레츠카,19PLS,CM,189,76,112,10,0.999830
8,4,E. 팔라시오스,20NG,CM,177,72,111,9,0.999767


In [ ]:
#끔찍한 혼종 추천리스트
filter(search_stat(using_df_list))

조건에 맞는 2806명의 선수를 검색합니다
동일선수 10명 제외


,우선순위,name,시즌,포지션,키,몸무게,스탯,강화,cos
0,2,사울,19UCL,CM,184,77,110,9,0.999862
1,2,레온 고레츠카,19TOTS,CM,189,79,111,9,0.999852
2,3,유상철,CAP,CM,184,78,112,7,0.999924
3,3,사울,20TOTS,CM,184,76,111,7,0.999783
4,3,루드 굴리트,NHD,CM,191,88,110,10,0.999754
5,3,루드 굴리트,TT,CM,191,88,109,9,0.999754
6,4,마리오 파샬리치,BOE21,CM,188,82,110,10,0.999981
7,4,루드 굴리트,HOT,CM,191,88,113,8,0.999754
8,4,슈테판 에펜베르크,CAP,CAM,188,88,109,5,0.999707
9,4,슈테판 에펜베르크,ICON,CM,188,88,112,5,0.999707
